# Crawl match instances and performance instances

In [147]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

driver = webdriver.Chrome()
#driver = webdriver.Safari()
#start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/Fixtures/Italy-Serie-A-2017-2018"
driver.get(start_url)

In [148]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")
        
    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_status = tds[2].find_element_by_tag_name("span").get_attribute("textContent")
                    if (match_status != "FT"):
                        continue
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

wait


In [153]:
# Crawl data of each match details
print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in tqdm(match_table.items()):
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    subdriver.get(match_link)
    time.sleep(1)
    web_match_trs = subdriver.find_elements_by_xpath("//*[@id=\"match-header\"]/table/tbody/tr")

    # Crawl basic match information
    web_match_tr = web_match_trs[0]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    home_team_id = int(re.findall(r'\d+', web_match_tds[0].find_element_by_tag_name("a").get_attribute("href"))[0])
    away_team_id = int(re.findall(r'\d+', web_match_tds[2].find_element_by_tag_name("a").get_attribute("href"))[0])

    web_match_tr = web_match_trs[1]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    web_match_divs = web_match_tds[1].find_elements_by_tag_name("div")
    match_datetime = web_match_divs[2].find_element_by_xpath("dl/dd[2]").text + ", " + web_match_divs[2].find_element_by_xpath("dl/dd[1]").text
    half_time_result = web_match_divs[1].find_element_by_xpath("dl/dd[1]").text
    result = web_match_divs[1].find_element_by_xpath("dl/dd[2]").text
    
    match_table[match_id]["match_datetime"] = match_datetime
    match_table[match_id]["home_team_id"] = home_team_id
    match_table[match_id]["away_team_id"] = away_team_id
    match_table[match_id]["half_time_result"] = half_time_result
    match_table[match_id]["result"] = result
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:273.
>>> 1: Crawling 1225358...
>>> 2: Crawling 1225233...
>>> 3: Crawling 1225444...
>>> 4: Crawling 1225392...
>>> 5: Crawling 1225247...
>>> 6: Crawling 1225338...
>>> 7: Crawling 1225371...
>>> 8: Crawling 1225337...
>>> 9: Crawling 1225223...
>>> 10: Crawling 1225339...
>>> 11: Crawling 1225222...
>>> 12: Crawling 1225397...
>>> 13: Crawling 1225162...
>>> 14: Crawling 1225366...
>>> 15: Crawling 1225218...
>>> 16: Crawling 1225201...
>>> 17: Crawling 1225334...
>>> 18: Crawling 1225393...
>>> 19: Crawling 1225285...
>>> 20: Crawling 1225309...
>>> 21: Crawling 1225297...
>>> 22: Crawling 1225390...
>>> 23: Crawling 1225272...
>>> 24: Crawling 1225231...
>>> 25: Crawling 1225262...
>>> 26: Crawling 1225260...
>>> 27: Crawling 1225188...
>>> 28: Crawling 1225329...
>>> 29: Crawling 1225380...
>>> 30: Crawling 1225313...
>>> 31: Crawling 1225373...
>>> 32: Crawling 1225229...
>>> 33: Crawling 1225182...
>>> 34: Crawling 1225195...
>>> 35: Crawling 1225306...


In [154]:
import json

with open('match_instances_table.json', 'w') as fp:
    json.dump(match_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [37]:
import pandas as pd
tmp = pd.read_json('match_instances_table.json', orient='index')

# Crawl team instances

In [155]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

driver = webdriver.Chrome()
#start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/TeamStatistics/Italy-Serie-A-2016-2017"
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/TeamStatistics/Italy-Serie-A-2017-2018"
driver.get(start_url)

In [156]:
team_table = {}

trs = driver.find_elements_by_xpath("//*[@id=\"top-team-stats-summary-content\"]/tr")

for tr in trs:
    team_link = tr.find_element_by_class_name("team-link").get_attribute("href")
    team_id = int(re.findall(r'\d+',team_link)[0])
    team_table[team_id] = {}
    team_table[team_id]["team_link"] = team_link

In [157]:
# Crawl data of each team
print("Total number of teams:{}.".format(len(team_table)))
cnt = 1
subdriver = webdriver.Chrome()
for team_id, team_details in tqdm(team_table.items()):
    team_link = team_details['team_link']
    # Change into the team page
    print(">>> {}: Crawling {}...".format(cnt, team_id))
    
    while True:
        try:
            subdriver.get(team_link)
            time.sleep(1)

            web_team_img_link = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/img").get_attribute("src")
            web_team_name = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/span").text
            break
        except:
            time.sleep(1)
    
    team_table[team_id]["team_img_link"] = web_team_img_link
    team_table[team_id]["team_name"] = web_team_name
    
    cnt += 1
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of teams:20.
>>> 1: Crawling 779...
>>> 2: Crawling 2889...
>>> 3: Crawling 278...
>>> 4: Crawling 71...
>>> 5: Crawling 72...
>>> 6: Crawling 73...
>>> 7: Crawling 75...
>>> 8: Crawling 300...
>>> 9: Crawling 77...
>>> 10: Crawling 78...
>>> 11: Crawling 271...
>>> 12: Crawling 80...
>>> 13: Crawling 76...
>>> 14: Crawling 84...
>>> 15: Crawling 86...
>>> 16: Crawling 87...
>>> 17: Crawling 267...
>>> 18: Crawling 276...
>>> 19: Crawling 1502...
>>> 20: Crawling 1503...
>>> finished.


In [158]:
import json

with open('team_instances_table.json', 'w') as fp:
    json.dump(team_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [68]:
import pandas as pd
tmp = pd.read_json('team_instances_table.json', orient='index')

# Crawl player instances

In [175]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

driver = webdriver.Chrome()
#start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/PlayerStatistics/Italy-Serie-A-2016-2017"
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/PlayerStatistics/Italy-Serie-A-2017-2018"
driver.get(start_url)

ALL_PLAYER = True

In [176]:
player_table = {}

if ALL_PLAYER:
    all_players_button = driver.find_element_by_xpath("//*[@id=\"apps\"]/dd[2]/a")
    all_players_button.click()

time.sleep(1)

In [177]:
next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
next_button_status = next_button.get_attribute("class")

player_cnt = 0
player_id_list = []
cnt = 1

while next_button_status != "option  disabled ":
    print(">>> Crawling page {}...".format(cnt))
    while True:
        try:
            player_statistic_tbody = driver.find_element_by_xpath("//*[@id=\"player-table-statistics-body\"]")
            player_trs = player_statistic_tbody.find_elements_by_xpath("tr")
            for tr in player_trs:
                pn = tr.find_element_by_class_name("pn")
                player_link = pn.find_element_by_class_name("player-link").get_attribute("href")
                player_id = int(re.findall(r'\d+', player_link)[0])

                player_table[player_id] = {}
                player_table[player_id]["player_link"] = player_link

                player_id_list.append(player_id)
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
    while True:
        try:
            next_button.click()
            break
        except:
            print(">>> error...waiting next click...")
            time.sleep(1)
            next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    cnt += 1
    next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    next_button_status = next_button.get_attribute("class")

print(">>> finished.")

>>> Crawling page 1...
>>> Crawling page 2...
>>> error...waiting next click...
>>> Crawling page 3...
>>> error...waiting next click...
>>> Crawling page 4...
>>> error...waiting next click...
>>> Crawling page 5...
>>> error...waiting next click...
>>> Crawling page 6...
>>> error...waiting next click...
>>> Crawling page 7...
>>> error...waiting next click...
>>> Crawling page 8...
>>> error...waiting next click...
>>> Crawling page 9...
>>> error...waiting next click...
>>> Crawling page 10...
>>> error...waiting next click...
>>> Crawling page 11...
>>> error...waiting next click...
>>> Crawling page 12...
>>> error...waiting next click...
>>> error...waiting next click...
>>> Crawling page 13...
>>> error...waiting next click...
>>> Crawling page 14...
>>> error...waiting next click...
>>> Crawling page 15...
>>> error...waiting next click...
>>> Crawling page 16...
>>> error...waiting next click...
>>> Crawling page 17...
>>> error...waiting next click...
>>> Crawling page 18...

In [178]:
# Crawl data of each player
print("Total number of players:{}.".format(len(player_table)))
cnt = 1
subdriver = webdriver.Chrome()
for player_id, player_details in tqdm(player_table.items()):
    player_link = player_details['player_link']
    # Change into the player page
    print(">>> {}: Crawling {}...".format(cnt, player_id))

    while True:
        try:
            subdriver.get(player_link)
            time.sleep(1)

            web_player_name = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[1]/dd").text
            web_player_img_link = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/img").get_attribute("src")
            web_player_birthday = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[1]/dd/i").text        
            try:
                web_player_height = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[2]/dd").text)[0])
            except:
                web_player_height = "N/A"
            try:
                web_player_weight = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[3]/dd").text)[0])
            except:
                web_player_weight = "N/A"
            team_id = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[2]/dd/a").get_attribute("href"))[0])
            no_dls = subdriver.find_elements_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl")
            web_player_position_items = no_dls[-1].find_elements_by_xpath("dd/ul/li")
            web_player_position = [item.text for item in web_player_position_items]
            
            player_table[player_id]["player_name"] = web_player_name   
            player_table[player_id]["player_img_link"] = web_player_img_link
            player_table[player_id]["player_birthday"] = web_player_birthday
            player_table[player_id]["player_height"] = web_player_height
            player_table[player_id]["player_weight"] = web_player_weight
            player_table[player_id]["team_id"] = team_id
            player_table[player_id]["player_position"] = web_player_position
            
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
        
    cnt += 1

print(">>> finished")
driver.close()
subdriver.close()   

Total number of players:502.
>>> 1: Crawling 141312...
>>> 2: Crawling 234286...
>>> 3: Crawling 68613...
>>> 4: Crawling 40966...
>>> 5: Crawling 109057...
>>> 6: Crawling 81725...
>>> 7: Crawling 323595...
>>> 8: Crawling 19471...
>>> 9: Crawling 22546...
>>> 10: Crawling 70675...
>>> 11: Crawling 327700...
>>> 12: Crawling 346136...
>>> 13: Crawling 106522...
>>> 14: Crawling 93211...
>>> 15: Crawling 135198...
>>> 16: Crawling 149509...
>>> 17: Crawling 78880...
>>> 18: Crawling 78881...
>>> error...waiting next trial...
>>> 19: Crawling 13348...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 20: Crawling 109062...
>>> 21: Crawling 149543...
>>> 22: Crawling 33834...
>>> 23: Crawling 142379...
>>> 24: Crawling 78892...
>>> 25: Crawling 15407...
>>> 26: Crawling 82994...
>>> 27: Crawling 129078...
>>> error...waiting next trial...
>>> 28: Crawling 320625...
>>> 29: Crawling 101437...
>>> 30: Crawling 24642...
>>> 31: Crawlin

>>> 212: Crawling 13764...
>>> 213: Crawling 128758...
>>> 214: Crawling 70087...
>>> 215: Crawling 135625...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 216: Crawling 106954...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 217: Crawling 100599...
>>> 218: Crawling 30635...
>>> error...waiting next trial...
>>> 219: Crawling 244301...
>>> 220: Crawling 10705...
>>> 221: Crawling 75218...
>>> 222: Crawling 284115...
>>> 223: Crawling 106964...
>>> 224: Crawling 141902...
>>> 225: Crawling 106967...
>>> 226: Crawling 106968...
>>> 227: Crawling 23001...
>>> 228: Crawling 344655...
>>> 229: Crawling 82399...
>>> 230: Crawling 300624...
>>> 231: Crawling 114147...
>>> 232: Crawling 13802...
>>> 233: Crawling 345682...
>>> 234: Crawling 332270...
>>> 235: Crawling 116207...
>>> 236: Crawling 312816...
>>> 237: Crawling 141813...
>>> 238: Crawling 25079...
>>> 239: Crawling 70141...
>>> 240: Crawling 7014

>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 440: Crawling 141169...
>>> 441: Crawling 125810...
>>> 442: Crawling 80757...
>>> 443: Crawling 123767...
>>> 444: Crawling 302968...
>>> 445: Crawling 105339...
>>> 446: Crawling 70524...
>>> 447: Crawling 30589...
>>> 448: Crawling 42878...
>>> 449: Crawling 42879...
>>> 450: Crawling 130368...
>>> 451: Crawling 82923...
>>> 452: Crawling 22406...
>>> 453: Crawling 33675...
>>> 454: Crawling 100962...
>>> 455: Crawling 121282...
>>> 456: Crawling 330643...
>>> 457: Crawling 302574...
>>> error...waiting next trial...
>>> 458: Crawling 86935...
>>> 459: Crawling 320408...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 460: Crawling 14831...
>>> 461: Crawling 71584...
>>> 462: Crawling 116641...
>>> error...waiting next trial...
>>> 463: Crawling 7323...
>>> 464: Crawling 294044...
>>> 465: Crawling 2987...
>>> 466: Crawling 320428...
>>> 467: Crawling 971

In [181]:
import json

with open('player_instances_table.json', 'w') as fp:
    json.dump(player_table, fp, indent=4)

# Crawl performance instances

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

driver = webdriver.Chrome()
#driver = webdriver.Safari()
start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
#start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/Fixtures/Italy-Serie-A-2017-2018"
driver.get(start_url)

In [2]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")

    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_status = tds[2].find_element_by_tag_name("span").get_attribute("textContent")
                    if (match_status != "FT"):
                        continue
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

In [3]:
# Crawl data of each match performance
performance_table = {}

print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in tqdm(match_table.items()):
    performance_table[match_id] = {}
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    while True:
        try:
            subdriver.get(match_link)
            time.sleep(1)
            
            # Record the match time first
            match_time = subdriver.find_element_by_xpath("//*[@id=\"match-centre-timeline\"]/div[1]/div[2]/div[2]/div[2]/div[2]/span").get_attribute("textContent")
            match_time = sum([int(digit) for digit in match_time.split("+")])
            
            # Change into player statistic page
            subdriver.get(subdriver.find_element_by_xpath("//*[@id=\"sub-sub-navigation\"]/ul/li[2]/a").get_attribute("href"))
            time.sleep(1)
            
            # Click through all player statistic tables such that they can be loaded
            # offensive button
            home_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[2]/a")
            home_offensive_button.click()
            time.sleep(2)
            away_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[2]/a")
            away_offensive_button.click()
            time.sleep(2)

            # defensive button
            home_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[3]/a")
            home_defensive_button.click()
            time.sleep(2)
            away_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[3]/a")
            away_defensive_button.click()
            time.sleep(2)
            
            # Passing button
            home_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[4]/a")
            home_passing_button.click()
            time.sleep(2)
            away_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[4]/a")
            away_passing_button.click()
            time.sleep(2)
            
            # Crawl match performance
            #home_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_offensive_trs = home_offensive_trs + away_offensive_trs

            for tr in total_offensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])
                performance_table[match_id][player_id] = {}

                #total_shots = tr.find_element_by_class_name("ShotsTotal ").text
                total_shots = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #ot_shots = tr.find_element_by_class_name("ShotOnTarget ").text
                ot_shots = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #key_passes = tr.find_element_by_class_name("KeyPassTotal ").text
                key_passes = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #dribbles = tr.find_element_by_class_name("DribbleWon ").text
                dribbles = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #foul_given = tr.find_element_by_class_name("FoulGiven ").text
                foul_given = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #offsides = tr.find_element_by_class_name("OffsideGiven ").text
                offsides = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #disp = tr.find_element_by_class_name("Dispossessed ").text
                disp = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #uns_touches = tr.find_element_by_class_name("Turnover ").text
                uns_touches = tr.find_element_by_xpath("td[11]").get_attribute("textContent")

                rating = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                key_events_icons = tr.find_elements_by_class_name("incident-icon")
                key_events = ["{},{},{}".format(icon.get_attribute("data-type"), icon.get_attribute("data-minute"), icon.get_attribute("data-second")) for icon in key_events_icons]

                if (rating == "-"):
                    minutes_played = 0
                else:
                    minutes_played = match_time

                for event in key_events:
                    event_type = event.split(",")[0]
                    event_minute = event.split(",")[1]
                    if (event_type == "18"):
                        minutes_played = int(event_minute)
                        break
                    elif (event_type == "19"):
                        minutes_played = match_time - int(event_minute)
                        break
                
                performance_table[match_id][player_id]["total_shots"] = re.sub('\t', '', total_shots)
                performance_table[match_id][player_id]["ot_shots"] = re.sub('\t', '', ot_shots)
                performance_table[match_id][player_id]["key_passes"] = re.sub('\t', '', key_passes)
                performance_table[match_id][player_id]["dribbles"] = re.sub('\t', '', dribbles)
                performance_table[match_id][player_id]["foul_given"] = re.sub('\t', '', foul_given)
                performance_table[match_id][player_id]["offsides"] = re.sub('\t', '', offsides)
                performance_table[match_id][player_id]["disp"] = re.sub('\t', '', disp)
                performance_table[match_id][player_id]["uns_touches"] = re.sub('\t', '', uns_touches)
                performance_table[match_id][player_id]["rating"] = rating
                performance_table[match_id][player_id]["key_events"] = key_events
                performance_table[match_id][player_id]["minutes_played"] = minutes_played

            #away_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #home_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_defensive_trs = home_defensive_trs + away_defensive_trs

            for tr in total_defensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_tackles = tr.find_element_by_class_name("TackleWonTotal ").text
                total_tackles = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #interception = tr.find_element_by_class_name("InterceptionAll ").text
                interception = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #clearances = tr.find_element_by_class_name("ClearanceTotal ").text
                clearances = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #blocked_shots = tr.find_element_by_class_name("ShotBlocked ").text
                blocked_shots = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #fouls_committed = tr.find_element_by_class_name("FoulCommitted ").text
                fouls_committed = tr.find_element_by_xpath("td[8]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_tackles"] = re.sub('\t', '', total_tackles)
                performance_table[match_id][player_id]["interception"] = re.sub('\t', '', interception)
                performance_table[match_id][player_id]["clearances"] = re.sub('\t', '', clearances)
                performance_table[match_id][player_id]["blocked_shots"] = re.sub('\t', '', blocked_shots)
                performance_table[match_id][player_id]["fouls_committed"] = re.sub('\t', '', fouls_committed)

            #home_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            total_passing_trs = home_passing_trs + away_passing_trs

            for tr in total_passing_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_passes = tr.find_element_by_class_name("TotalPasses ").text
                total_passes = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #pass_acc = tr.find_element_by_class_name("PassSuccessInMatch ").text
                pass_acc = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #crosses = tr.find_element_by_class_name("PassCrossTotal ").text
                crosses = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #acc_crosses = tr.find_element_by_class_name("PassCrossAccurate ").text
                acc_crosses = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #total_long_balls = tr.find_element_by_class_name("PassLongBallTotal ").text
                total_long_balls = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #acc_long_balls = tr.find_element_by_class_name("PassLongBallAccurate ").text
                acc_long_balls = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #total_through_balls = tr.find_element_by_class_name("PassThroughBallTotal ").text
                total_through_balls = tr.find_element_by_xpath("td[11]").get_attribute("textContent")
                #acc_through_balls = tr.find_element_by_class_name("PassThroughBallAccurate ").text
                acc_through_balls = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_passes"] = re.sub('\t', '', total_passes)
                performance_table[match_id][player_id]["pass_acc"] = re.sub('\t', '', pass_acc)
                performance_table[match_id][player_id]["crosses"] = re.sub('\t', '', crosses)
                performance_table[match_id][player_id]["acc_crosses"] = re.sub('\t', '', acc_crosses)
                performance_table[match_id][player_id]["total_long_balls"] = re.sub('\t', '', total_long_balls)
                performance_table[match_id][player_id]["acc_long_balls"] = re.sub('\t', '', acc_long_balls)
                performance_table[match_id][player_id]["total_through_balls"] = re.sub('\t', '', total_through_balls)
                performance_table[match_id][player_id]["acc_through_balls"] = re.sub('\t', '', acc_through_balls)
                if (len(performance_table[match_id][player_id]) < 24):
                    raise
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:380.
>>> 1: Crawling 1115239...
>>> 2: Crawling 1115275...
>>> 3: Crawling 1115220...
>>> 4: Crawling 1115375...
>>> 5: Crawling 1115216...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> error...waiting 

>>> 192: Crawling 1115455...
>>> 193: Crawling 1115217...
>>> 194: Crawling 1115462...
>>> 195: Crawling 1115280...
>>> 196: Crawling 1115360...
>>> 197: Crawling 1115521...
>>> 198: Crawling 1115149...
>>> error...waiting next trial...
>>> 199: Crawling 1115373...
>>> 200: Crawling 1115313...
>>> 201: Crawling 1115318...
>>> 202: Crawling 1115476...
>>> 203: Crawling 1115346...
>>> 204: Crawling 1115268...
>>> 205: Crawling 1115519...
>>> 206: Crawling 1115366...
>>> 207: Crawling 1115155...
>>> 208: Crawling 1115445...
>>> 209: Crawling 1115353...
>>> 210: Crawling 1115332...
>>> 211: Crawling 1115475...
>>> 212: Crawling 1115333...
>>> 213: Crawling 1115461...
>>> 214: Crawling 1115340...
>>> 215: Crawling 1115305...
>>> 216: Crawling 1115326...
>>> 217: Crawling 1115201...
>>> 218: Crawling 1115365...
>>> 219: Crawling 1115501...
>>> 220: Crawling 1115329...
>>> 221: Crawling 1115162...
>>> 222: Crawling 1115464...
>>> 223: Crawling 1115437...
>>> 224: Crawling 1115499...
>>> 225: 

In [186]:
import json

with open('performance_instances_table.json', 'w') as fp:
    json.dump(performance_table, fp, indent=4)

In [188]:
import json

with open('./2017-2018/performance_instances_table.json', 'r') as fp:
    data = json.load(fp)

In [202]:
for match, match_info in data.items():
    for player, player_info in match_info.items():
        if (len(player_info)< 24):
            print("match:{},player:{},len:{}".format(match,player,len(player_info)))

match:1225390,player:128967,len:19
match:1225390,player:9972,len:19
match:1225390,player:135625,len:19
match:1225390,player:353380,len:19
match:1225390,player:19112,len:19
match:1225390,player:44642,len:19
match:1225390,player:22377,len:19
match:1225390,player:34112,len:19
match:1225390,player:298060,len:19
match:1225390,player:106590,len:19
match:1225390,player:136279,len:19
match:1225390,player:87449,len:19
match:1225390,player:302918,len:19
match:1225390,player:317544,len:19
match:1225390,player:349848,len:19
match:1225390,player:322748,len:19
match:1225390,player:10372,len:19
match:1225390,player:125268,len:19
match:1225390,player:349847,len:19
match:1225390,player:27421,len:19
match:1225390,player:64271,len:19
match:1225288,player:106275,len:19
match:1225288,player:248109,len:19
match:1225288,player:106413,len:19
match:1225288,player:82923,len:19
match:1225288,player:84827,len:19
match:1225288,player:106963,len:19
match:1225288,player:19679,len:19
match:1225288,player:80443,len:19

In [208]:
data['1225362']['106275']

KeyError: '106275'